## Unzip The Raw Data

In [1]:
!./unzip.sh UCI_HAR_Dataset.zip 2>&1 > /dev/null

## Building The Dataset

In [2]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import L1L2
import numpy as np
import glob
import os

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

def load_y(subset):
    # Get the path
    path = f'UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/y_{subset}.txt'

    # Read the file
    y = np.loadtxt(path, delimiter=',', dtype=int)

    # # One-hot encode labels
    one_hot_labels = get_one_hot(y - 1 , len(np.unique(y)))
    if subset == 'train':
        assert one_hot_labels.shape == (7352, 6), f"Wrong dimensions: {one_hot_labels.shape} should be (7352, 6)"
    if subset == 'test':
        assert one_hot_labels.shape == (2947, 6), f"Wrong dimensions: {one_hot_labels.shape} should be (2947, 6)"
    assert y[0] - 1 == np.where(one_hot_labels[0] == np.max(one_hot_labels[0]))[0][0], f"Value mismatch {np.max(one_hot_labels[0])[0][0]} vs {y[13] - 1}"
    return one_hot_labels

def build_data(subset):
    if subset not in ['train', 'val', 'test']:
        raise Exception(f"Invalid subset: {subset}")

    folder_path = f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/Inertial Signals/"

    # Get all signal files in folder
    signal_files = glob.glob(os.path.join(folder_path, '*.txt'))
    # print(signal_files)

    assert len(signal_files) == 9, f"No signal files found in {folder_path}"
    signal_shape = np.loadtxt(signal_files[0]).shape
    # print(f"{signal_shape}")

    # Determine signal order based on file names
    signal_order = [
        "body_acc_x_",
        "body_acc_y_",
        "body_acc_z_",
        "body_gyro_x_",
        "body_gyro_y_",
        "body_gyro_z_",
        "total_acc_x_",
        "total_acc_y_",
        "total_acc_z_",
        ]

    # file_prefix = "UCI_HAR_Dataset/UCI_HAR_Dataset/train/Inertial Signals/"
    # file_suffix = ".txt"
    signal_files = [f"UCI_HAR_Dataset/UCI_HAR_Dataset/{subset}/Inertial Signals/{x}{subset}.txt" for x in signal_order]

    # Load signal data from each file and append to signals_data list
    signals_data = [np.loadtxt(x) for x in signal_files]

    # Transpose signal data array so that shape is (number of samples, number of timesteps, number of signals)
    signals_data = np.transpose(signals_data, (1, 2, 0))

    # Verify final shape of combined data
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    if subset == 'train':
        assert signals_data.shape == (7352, 128, len(signal_files))
    else:
        assert signals_data.shape == (2947, 128, len(signal_files))
    return signals_data

def load_data():
    return build_data('train'), load_y('train'), build_data('test'), load_y('test')

# Loading the train and test data
X_train, y_train, X_test, y_test = load_data()

In [3]:
first_sample = X_train[0]
first_timestep = first_sample[0]
assert len(first_sample) == 128
assert first_timestep[0] == 1.8085150e-004, print(first_timestep[0])
assert first_timestep[1] == 1.0766810e-002, print(first_timestep[1])

In [4]:
#function to count the number of classes
def count_classes(y):
    return len(set([tuple(category) for category in y]))

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = count_classes(y_train)

# Initializing parameters
n_epochs = 30
n_batch = 16

# Bias regularizer value - we will use elasticnet
regularizer = L1L2(0.01, 0.01)

print(f"Timesteps: {timesteps}")
print(f"Input dimention: {input_dim}")
print(f"Total samples: {len(X_train)}")

Timesteps: 128
Input dimention: 9
Total samples: 7352


In [5]:
# Model execution
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True, bias_regularizer=regularizer))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128, 64)           18944     
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 64)           256       
_________________________________________________________________
dropout (Dropout)            (None, 128, 64)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 48)                21696     
_________________________________________________________________
dropout_1 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 294       
Total params: 41,190
Trainable params: 41

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Training the model
model.fit(X_train, y_train, batch_size=n_batch, validation_data=(X_test, y_test), epochs=n_epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 42s 6ms/sample - loss: 1.8424 - acc: 0.6533 - val_loss: 1.3844 - val_acc: 0.6868
Epoch 2/30
7352/7352 [==============================] - 43s 6ms/sample - loss: 1.0107 - acc: 0.8124 - val_loss: 1.1238 - val_acc: 0.7248
Epoch 3/30
7352/7352 [==============================] - 44s 6ms/sample - loss: 0.6296 - acc: 0.8845 - val_loss: 0.5119 - val_acc: 0.8856
Epoch 4/30
7352/7352 [==============================] - 45s 6ms/sample - loss: 0.4249 - acc: 0.9010 - val_loss: 0.3619 - val_acc: 0.8867
Epoch 5/30
7352/7352 [==============================] - 44s 6ms/sample - loss: 0.3209 - acc: 0.9110 - val_loss: 0.3506 - val_acc: 0.8785
Epoch 6/30
7352/7352 [==============================] - 46s 6ms/sample - loss: 0.2395 - acc: 0.9245 - val_loss: 0.2257 - val_acc: 0.9179
Epoch 7/30
7352/7352 [==============

In [10]:
# Save model(s) for development purposes
import tensorflow as tf

# create a TFLiteConverter object
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# convert the model to TFLite format
tflite_model = converter.convert()

# save the TFLite model to a file
with open('./assets/model_lstm.tflite', 'wb') as f:
    f.write(tflite_model)

AttributeError: type object 'TFLiteConverter' has no attribute 'from_keras_model'